In [184]:
import pandas as pd
import ta 
import optuna 
import time
import numpy as np
from multiprocessing import Pool
from itertools import combinations, chain 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [185]:
data_1d = pd.read_csv("../data/aapl_1d_train.csv")
data_1d = data_1d.dropna()
data_1d.reset_index(drop=True, inplace=True)

In [186]:
data_1h = pd.read_csv("../data/aapl_1h_train.csv")
data_1h= data_1h.dropna()
data_1h.reset_index(drop=True, inplace=True)

In [187]:
data_5m = pd.read_csv("../data/aapl_5m_train.csv")
data_5m = data_5m.dropna()
data_5m.reset_index(drop=True, inplace=True)

In [188]:
data_1m = pd.read_csv("../data/aapl_1m_train.csv")
data_1m = data_1m.dropna()
data_1m.reset_index(drop=True, inplace=True)

In [189]:
def powerset(s):
    return chain.from_iterable(combinations(s,r) for r in range(1,len(s)+1))

In [190]:
def file_features(data, ds_type: str):
    data1=pd.DataFrame()
    #Calcular indicadores tecnicos
    cmf_data = ta.volume.ChaikinMoneyFlowIndicator(data.High, data.Low, data.Close, data.Volume, window = 14)
    rsi_data = ta.momentum.RSIIndicator(data.Close, window=14)
    
    data1["CMF"] = cmf_data.chaikin_money_flow()
    data1["RSI"] = rsi_data.rsi()
    # Calcular la volatilidad
    data1['Volatility'] = data['High'] - data['Low']
    data1['Close_Lag0'] = data['Close']
    # Calcular las tendencias
    for i in range(1, 5 + 1):
        data1[f'Close_Lag{i}'] = data['Close'].shift(i)
    #Variable ded respuesta
    if ds_type == "buy":
        data1['Response'] = (data['Close'] < data['Close'].shift(-10))
    else:
        data1['Response'] = (data['Close'] > data['Close'].shift(-10))
    
    data1 = data1.drop(data1.index[:30])
    data1 = data1.drop(data1.index[-30:])
    data1.reset_index(drop=True, inplace=True)
    
    return data1

In [191]:
dataresult_long_1d = file_features(data_1d, ds_type="buy")
dataresult_short_1d = file_features(data_1d, ds_type="sell")
dataresult_short_1d

,CMF,RSI,Volatility,Close_Lag0,Close_Lag1,Close_Lag2,Close_Lag3,Close_Lag4,Close_Lag5,Response
0,0.246933,61.064016,4.7488,119.0000,115.8080,108.7520,113.5008,112.6272,114.8784,False
1,0.223372,55.142449,6.0032,114.1280,119.0000,115.8080,108.7520,113.5008,112.6272,False
2,0.314459,55.852561,2.3856,114.8784,114.1280,119.0000,115.8080,108.7520,113.5008,False
3,0.171494,51.598684,4.4912,111.2496,114.8784,114.1280,119.0000,115.8080,108.7520,False
4,0.254775,54.250846,10.2480,113.8144,111.2496,114.8784,114.1280,119.0000,115.8080,False
...,...,...,...,...,...,...,...,...,...,...
5219,0.040511,45.850110,3.1900,119.4900,115.9700,116.3200,118.6900,119.0300,114.9500,True
5220,0.035327,45.534960,1.9600,119.2100,119.4900,115.9700,116.3200,118.6900,119.0300,True
5221,0.049474,45.606858,1.8000,119.2600,119.2100,119.4900,115.9700,116.3200,118.6900,True
5222,0.049731,47.169059,2.8400,120.3000,119.2600,119.2100,119.4900,115.9700,116.3200,False


In [192]:
dataresult_long_1h = file_features(data_1h, ds_type="buy")
dataresult_short_1h = file_features(data_1h, ds_type="sell")
dataresult_short_1h

,CMF,RSI,Volatility,Close_Lag0,Close_Lag1,Close_Lag2,Close_Lag3,Close_Lag4,Close_Lag5,Response
0,0.037057,42.198356,0.650001,119.830001,119.889999,121.150001,120.699996,120.955001,120.608398,True
1,0.110396,44.020547,1.010003,120.059997,119.830001,119.889999,121.150001,120.699996,120.955001,True
2,0.079499,44.875763,0.485000,120.165100,120.059997,119.830001,119.889999,121.150001,120.699996,True
3,0.007440,43.137629,0.790001,119.907699,120.165100,120.059997,119.830001,119.889999,121.150001,True
4,0.016513,37.447391,1.139999,118.970001,119.907699,120.165100,120.059997,119.830001,119.889999,True
...,...,...,...,...,...,...,...,...,...,...
315,0.230007,60.275954,0.309998,131.610000,131.755905,132.080001,132.068298,131.389999,131.733795,False
316,0.129291,55.880925,0.960006,130.960006,131.610000,131.755905,132.080001,132.068298,131.389999,False
317,0.148648,65.063275,2.080490,133.135604,130.960006,131.610000,131.755905,132.080001,132.068298,False
318,0.132043,59.049706,1.350006,132.147094,133.135604,130.960006,131.610000,131.755905,132.080001,False


In [193]:
dataresult_long_1m = file_features(data_1m, ds_type="buy")
dataresult_short_1m = file_features(data_1m, ds_type="sell")
dataresult_short_1m

,CMF,RSI,Volatility,Close_Lag0,Close_Lag1,Close_Lag2,Close_Lag3,Close_Lag4,Close_Lag5,Response
0,0.037057,42.198356,0.650001,119.830001,119.889999,121.150001,120.699996,120.955001,120.608398,True
1,0.110396,44.020547,1.010003,120.059997,119.830001,119.889999,121.150001,120.699996,120.955001,True
2,0.079499,44.875763,0.485000,120.165100,120.059997,119.830001,119.889999,121.150001,120.699996,True
3,0.007440,43.137629,0.790001,119.907699,120.165100,120.059997,119.830001,119.889999,121.150001,True
4,0.016513,37.447391,1.139999,118.970001,119.907699,120.165100,120.059997,119.830001,119.889999,True
...,...,...,...,...,...,...,...,...,...,...
315,0.230007,60.275954,0.309998,131.610000,131.755905,132.080001,132.068298,131.389999,131.733795,False
316,0.129291,55.880925,0.960006,130.960006,131.610000,131.755905,132.080001,132.068298,131.389999,False
317,0.148648,65.063275,2.080490,133.135604,130.960006,131.610000,131.755905,132.080001,132.068298,False
318,0.132043,59.049706,1.350006,132.147094,133.135604,130.960006,131.610000,131.755905,132.080001,False


In [194]:
dataresult_long_5m = file_features(data_5m, ds_type="buy")
dataresult_short_5m = file_features(data_5m, ds_type="sell")
dataresult_short_5m

,CMF,RSI,Volatility,Close_Lag0,Close_Lag1,Close_Lag2,Close_Lag3,Close_Lag4,Close_Lag5,Response
0,-0.313878,20.771997,0.519303,128.580001,128.820007,129.148498,129.413299,129.390106,129.690002,True
1,-0.377612,19.268881,0.440003,128.351699,128.580001,128.820007,129.148498,129.413299,129.390106,False
2,-0.449526,14.581731,0.849793,127.410003,128.351699,128.580001,128.820007,129.148498,129.413299,False
3,-0.284874,26.712155,1.269996,128.005004,127.410003,128.351699,128.580001,128.820007,129.148498,False
4,-0.293277,31.714234,0.649993,128.289993,128.005004,127.410003,128.351699,128.580001,128.820007,False
...,...,...,...,...,...,...,...,...,...,...
39095,-0.015916,49.213141,0.169907,128.779998,128.854995,128.955001,128.755004,128.860000,128.830001,True
39096,0.046092,49.526522,0.129898,128.789993,128.779998,128.854995,128.955001,128.755004,128.860000,True
39097,-0.087267,42.227222,0.340012,128.529998,128.789993,128.779998,128.854995,128.955001,128.755004,True
39098,-0.092697,36.251868,0.419998,128.259994,128.529998,128.789993,128.779998,128.854995,128.955001,True


In [195]:
def objective_log_regresor(trial, data):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Definir los parámetros a optimizar
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    C = trial.suggest_loguniform('C', 0.001, 1000)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])

    # Crear el modelo de regresión logística con los parámetros sugeridos
    model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=10_000, random_state=123)
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [196]:
def objective_svm(trial, data):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Definir los parámetros a optimizar
    C = trial.suggest_loguniform('C', 0.001, 1000)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
    else:
        degree = 3  # Valor predeterminado si el kernel no es 'poly'
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    # Crear el modelo SVM con los parámetros sugeridos
    model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, max_iter=100_000, random_state=123)
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [197]:
def objective_xgboost(trial, data):
    data = data.copy()
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # Definir los parámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, step=100)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
    subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1.0, 0.1)
    # Crear el modelo XGBoost con los parámetros sugeridos
    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=123
    )
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [198]:
def optimize_params_log_regresor(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_log_regresor(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=2)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [199]:
def optimize_params_svm(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_svm(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=2)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [200]:
def optimize_params_xgboost(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_xgboost(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=2)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [201]:
def optimize_params(data):
    # Optimización de regresión logística
    best_params_lr, best_accuracy_lr = optimize_params_log_regresor(data)
    print("Mejores parámetros de regresión logística:", best_params_lr)
    print("Precisión del modelo de regresión logística:", best_accuracy_lr)
    # Optimización de SVM
    best_params_svm, best_accuracy_svm = optimize_params_svm(data)
    print("Mejores parámetros de SVM:", best_params_svm)
    print("Precisión del modelo de SVM:", best_accuracy_svm)
    # Optimización de XGBoost
    best_params_xgb, best_accuracy_xgb = optimize_params_xgboost(data)
    print("Mejores parámetros de XGBoost:", best_params_xgb)
    print("Precisión del modelo de XGBoost:", best_accuracy_xgb)

In [202]:
params_1d_long = optimize_params(dataresult_long_1d)
params_1d_short = optimize_params(dataresult_short_1d)

[I 2024-03-08 10:44:01,570] A new study created in memory with name: no-name-d1ad5ceb-53ee-4ffc-b4d2-5f34e62fa553
[I 2024-03-08 10:44:05,589] Trial 0 finished with value: 0.6564593301435406 and parameters: {'penalty': 'l1', 'C': 338.8347276808977, 'solver': 'saga'}. Best is trial 0 with value: 0.6564593301435406.
[I 2024-03-08 10:44:09,434] Trial 1 finished with value: 0.6564593301435406 and parameters: {'penalty': 'l1', 'C': 120.29526600413703, 'solver': 'saga'}. Best is trial 0 with value: 0.6564593301435406.
[I 2024-03-08 10:44:09,435] A new study created in memory with name: no-name-49c9cf16-ae28-495e-94c4-6df7df1cef3f


Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 338.8347276808977, 'solver': 'saga'}
Precisión del modelo de regresión logística: 0.6564593301435406


[I 2024-03-08 10:44:10,307] Trial 0 finished with value: 0.6593301435406699 and parameters: {'C': 0.0016060352833009584, 'kernel': 'linear'}. Best is trial 0 with value: 0.6593301435406699.
[I 2024-03-08 10:44:10,670] Trial 1 finished with value: 0.6593301435406699 and parameters: {'C': 4.936418357583564, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.6593301435406699.
[I 2024-03-08 10:44:10,670] A new study created in memory with name: no-name-dcb95bd2-114d-4014-816d-d9d4cf702d9b


Mejores parámetros de SVM: {'C': 0.0016060352833009584, 'kernel': 'linear'}
Precisión del modelo de SVM: 0.6593301435406699


[I 2024-03-08 10:44:10,982] Trial 0 finished with value: 0.5712918660287082 and parameters: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.20497189453932135, 'subsample': 0.8, 'colsample_bytree': 0.5}. Best is trial 0 with value: 0.5712918660287082.
[I 2024-03-08 10:44:11,734] Trial 1 finished with value: 0.5684210526315789 and parameters: {'n_estimators': 900, 'max_depth': 8, 'learning_rate': 0.04242629295608167, 'subsample': 0.9, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.5712918660287082.
[I 2024-03-08 10:44:11,735] A new study created in memory with name: no-name-d3a860a6-c347-4268-97cf-7faf174f41a3


...

Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 505.00639904258753, 'solver': 'liblinear'}
Precisión del modelo de regresión logística: 0.6602870813397129


[I 2024-03-08 10:44:17,353] Trial 0 finished with value: 0.661244019138756 and parameters: {'C': 0.0019435907131005682, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 0 with value: 0.661244019138756.
[I 2024-03-08 10:44:18,128] Trial 1 finished with value: 0.661244019138756 and parameters: {'C': 9.751654044428191, 'kernel': 'poly', 'degree': 2, 'gamma': 'scale'}. Best is trial 0 with value: 0.661244019138756.
[I 2024-03-08 10:44:18,128] A new study created in memory with name: no-name-200af927-8397-4615-bbf5-db6355ffc967
[I 2024-03-08 10:44:18,287] Trial 0 finished with value: 0.569377990430622 and parameters: {'n_estimators': 600, 'max_depth': 3, 'learning_rate': 0.08087896167042405, 'subsample': 0.8, 'colsample_bytree': 0.5}. Best is trial 0 with value: 0.569377990430622.


Mejores parámetros de SVM: {'C': 0.0019435907131005682, 'kernel': 'sigmoid', 'gamma': 'scale'}
Precisión del modelo de SVM: 0.661244019138756


[I 2024-03-08 10:44:18,981] Trial 1 finished with value: 0.5492822966507177 and parameters: {'n_estimators': 800, 'max_depth': 8, 'learning_rate': 0.016854675011246666, 'subsample': 1.0, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.569377990430622.


Mejores parámetros de XGBoost: {'n_estimators': 600, 'max_depth': 3, 'learning_rate': 0.08087896167042405, 'subsample': 0.8, 'colsample_bytree': 0.5}
Precisión del modelo de XGBoost: 0.569377990430622


In [203]:
params_1h_long = optimize_params(dataresult_long_1h)
params_1h_short = optimize_params(dataresult_short_1h)

[I 2024-03-08 10:44:18,987] A new study created in memory with name: no-name-5d03c708-8ca4-4bb2-8bfb-599fb8946f3a
[I 2024-03-08 10:44:18,993] Trial 0 finished with value: 0.234375 and parameters: {'penalty': 'l1', 'C': 0.0014278079128057035, 'solver': 'liblinear'}. Best is trial 0 with value: 0.234375.
[I 2024-03-08 10:44:18,997] Trial 1 finished with value: 0.234375 and parameters: {'penalty': 'l1', 'C': 0.006416180832892937, 'solver': 'liblinear'}. Best is trial 0 with value: 0.234375.
[I 2024-03-08 10:44:18,998] A new study created in memory with name: no-name-c111216b-0062-4a0a-b39a-ce84712b57f6
[I 2024-03-08 10:44:19,006] Trial 0 finished with value: 0.234375 and parameters: {'C': 0.0016083695514863604, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.234375.
[I 2024-03-08 10:44:19,013] Trial 1 finished with value: 0.234375 and parameters: {'C': 0.003387734982153313, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.234375.
[I 2024-03-08 10:44:19,014

Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 0.0014278079128057035, 'solver': 'liblinear'}
Precisión del modelo de regresión logística: 0.234375
Mejores parámetros de SVM: {'C': 0.0016083695514863604, 'kernel': 'rbf', 'gamma': 'auto'}
Precisión del modelo de SVM: 0.234375


[I 2024-03-08 10:44:19,204] Trial 0 finished with value: 0.40625 and parameters: {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.050090043255527315, 'subsample': 0.6, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.40625.
[I 2024-03-08 10:44:19,444] Trial 1 finished with value: 0.390625 and parameters: {'n_estimators': 900, 'max_depth': 5, 'learning_rate': 0.05676849407675641, 'subsample': 0.6, 'colsample_bytree': 0.7}. Best is trial 0 with value: 0.40625.
[I 2024-03-08 10:44:19,445] A new study created in memory with name: no-name-75f82ac8-3ccf-42fc-880a-733615160b9d
[I 2024-03-08 10:44:19,453] Trial 0 finished with value: 0.78125 and parameters: {'penalty': 'l1', 'C': 0.001101925237033472, 'solver': 'saga'}. Best is trial 0 with value: 0.78125.
[I 2024-03-08 10:44:19,629] Trial 1 finished with value: 0.328125 and parameters: {'penalty': 'l2', 'C': 0.31223845581324766, 'solver': 'saga'}. Best is trial 0 with value: 0.78125.
[I 2024-03-08 10:44:19,630] A new study crea

Mejores parámetros de XGBoost: {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.050090043255527315, 'subsample': 0.6, 'colsample_bytree': 0.9}
Precisión del modelo de XGBoost: 0.40625
Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 0.001101925237033472, 'solver': 'saga'}
Precisión del modelo de regresión logística: 0.78125
Mejores parámetros de SVM: {'C': 3.1829337194198657, 'kernel': 'rbf', 'gamma': 'scale'}
Precisión del modelo de SVM: 0.21875


[I 2024-03-08 10:44:19,713] Trial 0 finished with value: 0.4375 and parameters: {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.04533156402768964, 'subsample': 0.5, 'colsample_bytree': 0.5}. Best is trial 0 with value: 0.4375.
[I 2024-03-08 10:44:20,457] Trial 1 finished with value: 0.359375 and parameters: {'n_estimators': 600, 'max_depth': 7, 'learning_rate': 0.022793135399056988, 'subsample': 1.0, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.4375.


Mejores parámetros de XGBoost: {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.04533156402768964, 'subsample': 0.5, 'colsample_bytree': 0.5}
Precisión del modelo de XGBoost: 0.4375


In [204]:
params_1m_long = optimize_params(dataresult_long_1m)
params_1m_short = optimize_params(dataresult_short_1m)

[I 2024-03-08 10:44:20,464] A new study created in memory with name: no-name-39e0a670-dc2d-472e-a3a0-b76cdaac96f0
[I 2024-03-08 10:44:20,480] Trial 0 finished with value: 0.3125 and parameters: {'penalty': 'l2', 'C': 8.463289162084212, 'solver': 'liblinear'}. Best is trial 0 with value: 0.3125.
[I 2024-03-08 10:44:20,486] Trial 1 finished with value: 0.359375 and parameters: {'penalty': 'l2', 'C': 0.010405629107406833, 'solver': 'liblinear'}. Best is trial 1 with value: 0.359375.
[I 2024-03-08 10:44:20,487] A new study created in memory with name: no-name-6e01a354-28b8-4f18-89ba-1977ea04232d
[I 2024-03-08 10:44:20,495] Trial 0 finished with value: 0.234375 and parameters: {'C': 0.02891729617292502, 'kernel': 'linear'}. Best is trial 0 with value: 0.234375.
[I 2024-03-08 10:44:20,502] Trial 1 finished with value: 0.234375 and parameters: {'C': 0.005587796621533931, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.234375.
[I 2024-03-08 10:44:20,503] A new study create

Mejores parámetros de regresión logística: {'penalty': 'l2', 'C': 0.010405629107406833, 'solver': 'liblinear'}
Precisión del modelo de regresión logística: 0.359375
Mejores parámetros de SVM: {'C': 0.02891729617292502, 'kernel': 'linear'}
Precisión del modelo de SVM: 0.234375


[I 2024-03-08 10:44:20,769] Trial 0 finished with value: 0.375 and parameters: {'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.02572836520302602, 'subsample': 0.8, 'colsample_bytree': 1.0}. Best is trial 0 with value: 0.375.
[I 2024-03-08 10:44:20,922] Trial 1 finished with value: 0.359375 and parameters: {'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.10180470895971654, 'subsample': 0.9, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.375.
[I 2024-03-08 10:44:20,922] A new study created in memory with name: no-name-5f318273-f8cd-4612-86a6-ff0b18a05acc
[I 2024-03-08 10:44:20,927] Trial 0 finished with value: 0.78125 and parameters: {'penalty': 'l1', 'C': 0.002778605059659017, 'solver': 'saga'}. Best is trial 0 with value: 0.78125.
[I 2024-03-08 10:44:20,932] Trial 1 finished with value: 0.3125 and parameters: {'penalty': 'l2', 'C': 18.71856251277501, 'solver': 'liblinear'}. Best is trial 0 with value: 0.78125.
[I 2024-03-08 10:44:20,933] A new study created i

Mejores parámetros de XGBoost: {'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.02572836520302602, 'subsample': 0.8, 'colsample_bytree': 1.0}
Precisión del modelo de XGBoost: 0.375
Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 0.002778605059659017, 'solver': 'saga'}
Precisión del modelo de regresión logística: 0.78125
Mejores parámetros de SVM: {'C': 0.28120622947860485, 'kernel': 'rbf', 'gamma': 'auto'}
Precisión del modelo de SVM: 0.78125


[I 2024-03-08 10:44:21,325] Trial 1 finished with value: 0.359375 and parameters: {'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.30721293075791994, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 0 with value: 0.40625.


Mejores parámetros de XGBoost: {'n_estimators': 400, 'max_depth': 3, 'learning_rate': 0.11719371112820326, 'subsample': 0.7, 'colsample_bytree': 0.8}
Precisión del modelo de XGBoost: 0.40625


In [205]:
params_5m_long = optimize_params(dataresult_long_5m)
params_5m_short = optimize_params(dataresult_short_5m)

[I 2024-03-08 10:44:21,330] A new study created in memory with name: no-name-d19ecfed-b938-4a11-b21e-f7509a778a5a
[I 2024-03-08 10:44:25,087] Trial 0 finished with value: 0.4966751918158568 and parameters: {'penalty': 'l2', 'C': 1.8666504477996768, 'solver': 'saga'}. Best is trial 0 with value: 0.4966751918158568.
[I 2024-03-08 10:50:01,508] Trial 1 finished with value: 0.4989769820971867 and parameters: {'penalty': 'l1', 'C': 4.043049203980127, 'solver': 'liblinear'}. Best is trial 1 with value: 0.4989769820971867.
[I 2024-03-08 10:50:01,511] A new study created in memory with name: no-name-f02d0604-6b5a-456f-aa7c-78294eff92bb


Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 4.043049203980127, 'solver': 'liblinear'}
Precisión del modelo de regresión logística: 0.4989769820971867


[I 2024-03-08 10:50:45,632] Trial 0 finished with value: 0.5023017902813299 and parameters: {'C': 29.24008590889724, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.5023017902813299.
[I 2024-03-08 10:50:59,725] Trial 1 finished with value: 0.5006393861892583 and parameters: {'C': 382.68098832113617, 'kernel': 'poly', 'degree': 3, 'gamma': 'auto'}. Best is trial 0 with value: 0.5023017902813299.
[I 2024-03-08 10:50:59,726] A new study created in memory with name: no-name-e19313df-f43d-4098-aa1a-0fcd28e695ce


Mejores parámetros de SVM: {'C': 29.24008590889724, 'kernel': 'rbf', 'gamma': 'scale'}
Precisión del modelo de SVM: 0.5023017902813299


[I 2024-03-08 10:51:00,209] Trial 0 finished with value: 0.49539641943734014 and parameters: {'n_estimators': 700, 'max_depth': 3, 'learning_rate': 0.010616472415453023, 'subsample': 0.5, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.49539641943734014.
[I 2024-03-08 10:51:00,354] Trial 1 finished with value: 0.4971867007672634 and parameters: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.11290588406986976, 'subsample': 0.6, 'colsample_bytree': 0.7}. Best is trial 1 with value: 0.4971867007672634.
[I 2024-03-08 10:51:00,355] A new study created in memory with name: no-name-b38a8520-f957-455b-a633-53146c02d216


...

Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 0.20108432606818857, 'solver': 'liblinear'}
Precisión del modelo de regresión logística: 0.5025575447570333


[I 2024-03-08 10:54:17,521] Trial 0 finished with value: 0.5046035805626599 and parameters: {'C': 3.23470738361756, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.5046035805626599.
[I 2024-03-08 10:54:59,574] Trial 1 finished with value: 0.5019181585677749 and parameters: {'C': 0.002864880934479293, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.5046035805626599.
[I 2024-03-08 10:54:59,582] A new study created in memory with name: no-name-5b367e4e-f138-4e14-b54d-a14069704664


Mejores parámetros de SVM: {'C': 3.23470738361756, 'kernel': 'rbf', 'gamma': 'scale'}
Precisión del modelo de SVM: 0.5046035805626599


[I 2024-03-08 10:55:00,418] Trial 0 finished with value: 0.5010230179028133 and parameters: {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.030360926232124368, 'subsample': 0.6, 'colsample_bytree': 0.7}. Best is trial 0 with value: 0.5010230179028133.
[I 2024-03-08 10:55:02,060] Trial 1 finished with value: 0.5016624040920716 and parameters: {'n_estimators': 700, 'max_depth': 9, 'learning_rate': 0.0512583014964379, 'subsample': 0.9, 'colsample_bytree': 0.9}. Best is trial 1 with value: 0.5016624040920716.


Mejores parámetros de XGBoost: {'n_estimators': 700, 'max_depth': 9, 'learning_rate': 0.0512583014964379, 'subsample': 0.9, 'colsample_bytree': 0.9}
Precisión del modelo de XGBoost: 0.5016624040920716


In [225]:
def buy_signals(data):
    buy_signals = pd.DataFrame()
    # Selecciona las características
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]

    # Crear modelos con los mejores parámetros encontrados para cada algoritmo
    best_logistic_model = LogisticRegression(penalty='l2', C= 0.010405629107406833, solver= 'liblinear')
    best_svm_model = SVC(C=0.02891729617292502, kernel= 'linear', gamma='scale')
    best_xgboost_model = XGBClassifier(n_estimators=300, max_depth=9, learning_rate=0.02572836520302602, subsample=0.8, colsample_bytree=1.0)

    # Entrenar los modelos con todo el conjunto de datos original
    best_logistic_model.fit(X, y)
    best_svm_model.fit(X, y)
    best_xgboost_model.fit(X, y)

    # Realizar predicciones en el conjunto de datos original
    predictions_lr = best_logistic_model.predict(X)
    predictions_svm = best_svm_model.predict(X)
    predictions_xgboost = best_xgboost_model.predict(X)

    # Agregar las predicciones como nuevas columnas al conjunto de datos original
    buy_signals['predicciones_lr'] = predictions_lr
    buy_signals['predicciones_svm'] = predictions_svm
    buy_signals['predicciones_xgboost'] = predictions_xgboost
    
    return buy_signals

In [226]:
def sell_signals(data):
    sell_signals = pd.DataFrame()
    # Selecciona las características
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]

    # Crear modelos con los mejores parámetros encontrados para cada algoritmo
    best_logistic_model = LogisticRegression(penalty= 'l1', C= 0.002778605059659017, solver= 'saga')
    best_svm_model = SVC(C= 0.28120622947860485, kernel= 'rbf', gamma= 'auto')
    best_xgboost_model = XGBClassifier(n_estimators= 400, max_depth= 3, learning_rate= 0.11719371112820326, subsample= 0.7, colsample_bytree= 0.8)

    # Entrenar los modelos con todo el conjunto de datos original
    best_logistic_model.fit(X, y)
    best_svm_model.fit(X, y)
    best_xgboost_model.fit(X, y)

    # Realizar predicciones en el conjunto de datos original
    predictions_lr = best_logistic_model.predict(X)
    predictions_svm = best_svm_model.predict(X)
    predictions_xgboost = best_xgboost_model.predict(X)
    predictions_xgboost_bool = predictions_xgboost.astype(bool)

    # Agregar las predicciones como nuevas columnas al conjunto de datos original
    sell_signals['predicciones_lr'] = predictions_lr
    sell_signals['predicciones_svm'] = predictions_svm
    sell_signals['predicciones_xgboost'] = predictions_xgboost_bool
    
    return sell_signals

In [227]:
global_buy_signals = buy_signals(dataresult_long_1d)
global_sell_signals = sell_signals(dataresult_short_1d)

In [228]:
global_buy_signals = buy_signals(dataresult_long_1h)
global_sell_signals = sell_signals(dataresult_short_1h)

In [229]:
global_buy_signals = buy_signals(dataresult_long_1m)
global_sell_signals = sell_signals(dataresult_short_1m)

In [230]:
global_buy_signals = buy_signals(dataresult_long_5m)
global_sell_signals = sell_signals(dataresult_short_5m)

In [231]:
def backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares):
    history = []
    active_operations = []
    cash = 1_000_000
    com = 1.25 / 100
    
    for i, row in data.iterrows():
        # close active operation
        active_op_temp = []
        for operation in active_operations:
            if operation["stop_loss"] > row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
            elif operation["take_profit"] < row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
            else:
                active_op_temp.append(operation)
        active_operations = active_op_temp

        # check if we have enough cash
        if cash < (row.Close * (1 + com)):
            asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
            portfolio_value = cash + asset_vals
            continue

        # Apply buy signals
        if buy_signals.loc[i].any():
            active_operations.append({
                "bought": row.Close,
                "n_shares": n_shares,
                "stop_loss": row.Close * stop_loss,
                "take_profit": row.Close * take_profit
            })

            cash -= row.Close * (1 + com) * n_shares

        # Apply sell signals
        if sell_signals.loc[i].any():
            active_op_temp = []
            for operation in active_operations:
                if operation["take_profit"] < row.Close or operation["stop_loss"] > row.Close:
                    cash += (row.Close * operation["n_shares"]) * (1 - com)
                else:
                    active_op_temp.append(operation)
            active_operations = active_op_temp

        asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
        portfolio_value = cash + asset_vals

    return portfolio_value, active_operations

In [232]:
def optimize(trial, strategy, data):
    portfolio_value = 0

    stop_loss = trial.suggest_float("stop_loss", 0.00250, 0.05)
    take_profit = trial.suggest_float("take_profit", 0.00250, 0.05)
    n_shares = trial.suggest_int("n_shares", 5, 200)

    strat_params = {}

    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()

    if "logistic" in strategy:
        buy_signals["logistic"] = global_buy_signals["predicciones_lr"]
        sell_signals["logistic"] = global_sell_signals["predicciones_lr"]
        
    if "svm" in strategy:
        buy_signals["svm"] = global_buy_signals["predicciones_svm"]
        sell_signals["svm"] = global_sell_signals["predicciones_svm"]
        
    if "xg" in strategy:
        buy_signals["xg"] = global_buy_signals["predicciones_xgboost"]
        sell_signals["xg"] = global_sell_signals["predicciones_xgboost"]
    
    return backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares)

In [233]:
def optimize_file(data):
    data = data.drop(data.index[:30])
    data = data.drop(data.index[-30:])
    data.reset_index(drop=True, inplace=True)
    strategies = list(powerset(["logistic", "svm", "xg"]))
    best_strat = None
    best_val = -1
    best_params = None

    for strat in strategies:
        study = optuna.create_study(direction="maximize")
        study.optimize(lambda x: optimize(x, strat, data), n_trials=30)
        value = study.best_value
        if value > best_val:
            best_val = value
            best_strat = strat
            best_params = study.best_params
    print(study.best_value)
    print(best_strat)
    print(best_params)

    return {"file": data,
            "strat": best_strat,
            "value": best_val,
            "params": best_params}

In [234]:
file_1d = optimize_file(data_1d)

[I 2024-03-08 11:34:47,706] A new study created in memory with name: no-name-9018e3be-9ed0-44a5-8db8-982aba8c0406
[I 2024-03-08 11:34:48,010] Trial 0 finished with value: -302.7069349952144 and parameters: {'stop_loss': 0.017850360295861015, 'take_profit': 0.04057151235380353, 'n_shares': 81}. Best is trial 0 with value: -302.7069349952144.
[I 2024-03-08 11:34:48,271] Trial 1 finished with value: -549.5401999957248 and parameters: {'stop_loss': 0.033440630143553134, 'take_profit': 0.03576606269950994, 'n_shares': 120}. Best is trial 0 with value: -302.7069349952144.
[I 2024-03-08 11:34:48,524] Trial 2 finished with value: -1096.9311349975178 and parameters: {'stop_loss': 0.03675929350217185, 'take_profit': 0.04128560917752846, 'n_shares': 141}. Best is trial 0 with value: -302.7069349952144.
[I 2024-03-08 11:34:48,893] Trial 3 finished with value: -15781.457825002613 and parameters: {'stop_loss': 0.03177687439367268, 'take_profit': 0.04664936324569879, 'n_shares': 45}. Best is trial 0 

884482.7848749985
('xg',)
{'stop_loss': 0.04867061072737812, 'take_profit': 0.014701384261391554, 'n_shares': 5}


In [235]:
file_1m = optimize_file(data_1m)


[I 2024-03-08 11:36:00,456] A new study created in memory with name: no-name-10b45085-63cc-424d-9653-b48050a90862
[I 2024-03-08 11:36:00,484] Trial 0 finished with value: 942566.5413551113 and parameters: {'stop_loss': 0.031120649011963812, 'take_profit': 0.00941684642825925, 'n_shares': 63}. Best is trial 0 with value: 942566.5413551113.
[I 2024-03-08 11:36:00,511] Trial 1 finished with value: 941654.899154401 and parameters: {'stop_loss': 0.049065783145287804, 'take_profit': 0.021569125310401917, 'n_shares': 64}. Best is trial 0 with value: 942566.5413551113.
[I 2024-03-08 11:36:00,537] Trial 2 finished with value: 854137.2478860044 and parameters: {'stop_loss': 0.017135351787595756, 'take_profit': 0.04473201226150637, 'n_shares': 160}. Best is trial 0 with value: 942566.5413551113.
[I 2024-03-08 11:36:00,562] Trial 3 finished with value: 873281.7341009628 and parameters: {'stop_loss': 0.009308619112717421, 'take_profit': 0.03620115061755097, 'n_shares': 139}. Best is trial 0 with va

995246.8219530623
('xg',)
{'stop_loss': 0.049549981034627176, 'take_profit': 0.0030567784866075516, 'n_shares': 6}


In [236]:
file_5m = optimize_file(data_5m)


[I 2024-03-08 11:36:07,253] A new study created in memory with name: no-name-6ba79a4c-19ea-41ff-a05e-6a6d3c62827f
[I 2024-03-08 11:36:08,214] Trial 0 finished with value: -20.859335220751746 and parameters: {'stop_loss': 0.046326465841431484, 'take_profit': 0.04661269868349891, 'n_shares': 91}. Best is trial 0 with value: -20.859335220751746.
[I 2024-03-08 11:36:09,273] Trial 1 finished with value: -42.55790216963942 and parameters: {'stop_loss': 0.0237097743982345, 'take_profit': 0.005817024099149145, 'n_shares': 194}. Best is trial 0 with value: -20.859335220751746.
[I 2024-03-08 11:36:10,133] Trial 2 finished with value: -117.66601759778132 and parameters: {'stop_loss': 0.04504132280723247, 'take_profit': 0.044507587337527896, 'n_shares': 192}. Best is trial 0 with value: -20.859335220751746.
[I 2024-03-08 11:36:11,535] Trial 3 finished with value: 114.24215238680472 and parameters: {'stop_loss': 0.03831251576786524, 'take_profit': 0.035868801141761206, 'n_shares': 26}. Best is tria

278475.25759626424
('xg',)
{'stop_loss': 0.020799244207322022, 'take_profit': 0.049844129192893015, 'n_shares': 5}


In [237]:
file_1h = optimize_file(data_1h)

[I 2024-03-08 11:41:20,063] A new study created in memory with name: no-name-639fdd6f-93c9-4cd2-9dd0-8e3cd8565f90
[I 2024-03-08 11:41:20,091] Trial 0 finished with value: 915217.2753337375 and parameters: {'stop_loss': 0.028987092005307382, 'take_profit': 0.004307060665200486, 'n_shares': 93}. Best is trial 0 with value: 915217.2753337375.
[I 2024-03-08 11:41:20,118] Trial 1 finished with value: 982678.7981864631 and parameters: {'stop_loss': 0.002843367854976402, 'take_profit': 0.045620493339248364, 'n_shares': 19}. Best is trial 1 with value: 982678.7981864631.
[I 2024-03-08 11:41:20,145] Trial 2 finished with value: 922510.4129394366 and parameters: {'stop_loss': 0.00589461618418733, 'take_profit': 0.04230461941109366, 'n_shares': 85}. Best is trial 1 with value: 982678.7981864631.
[I 2024-03-08 11:41:20,172] Trial 3 finished with value: 981767.1559857475 and parameters: {'stop_loss': 0.0384603144640915, 'take_profit': 0.014347170113478833, 'n_shares': 20}. Best is trial 1 with valu

995246.8219530623
('xg',)
{'stop_loss': 0.049817744389634114, 'take_profit': 0.03716289935426518, 'n_shares': 5}
